## Packages

In [ ]:
import scipy.io as sio 
import numpy as np
from plot_conf_mat import plot_confusion_matrix
import matplotlib.pyplot as plt

## Import data from matlab

In [ ]:
fichier = sio.loadmat('fichier.mat') # Download the features/labels generated by the matlab script
Sequences_arduino = sio.loadmat('input/sequences_arduino.mat') #Download the sequences correposnding

label = fichier['Labels']
data_fourier = fichier['Data_fourier']
sequences_arduino = Sequences_arduino['sequences'].reshape(90,)

## Separate the True and False labels

In [ ]:
var_D = np.where(label == 1) # Find the 1 corresponding
index_true = var_D[0]
len_index = index_true.shape
num_index_true = int (len_index[0])

data_no_p300 = np.delete(data_fourier, index_true, axis = 0) # ALl the features corresponding to 0

data_p300 = np.zeros([num_index_true, 320])# All the features corresponding to 1
for i in range(num_index_true):
    data_p300[i] = data_fourier[index_true[i],:]

## Create the data set

In [ ]:
X = np.append(data_p300, data_no_p300[0:num_index_true,:],axis=0) # Create set of the same size
y = np.append(np.ones([num_index_true, 1]), np.zeros([num_index_true, 1]),axis=0)

## Reduce the dimension of the problem using PCA

In [ ]:
from sklearn.decomposition import RandomizedPCA
n_components_PCA = 80
pca = RandomizedPCA(n_components=n_components_PCA, random_state= 7).fit(X)
X_pca = pca.transform(X)

## Training / Test set

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.4, random_state=3)

## SVM Classifier

In [ ]:
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV

#Find the best parameter
param_grid = {'C': [ 0.1, 1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1]}  

clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
y_train = y_train.reshape(int(y_train.shape[0]),)
clf = clf.fit(X_train, y_train)  

## Accuracy and Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_test)
binary_acc = round(accuracy_score(y_test,y_pred),3)
conf_mat = confusion_matrix(y_test,y_pred, [0, 1] )
plot_confusion_matrix(conf_mat, classes = [0,1],normalize=False, title='Confusion matrix',cmap=plt.cm.Blues)

## Predict the word of the user

In [ ]:
from guess_combination import predict_word

predicted_combination = predict_word(0, 19, data_fourier, sequences_arduino, pca, clf)

## Assess the accuracy to predict the good word

In [ ]:
patient_combination = [1, 1, 4, 2, 4, 5, 7, 2, 9, 4, 5, 8, 9, 8, 3, 8, 8, 5, 4]

acc = round(accuracy_score(predicted_combination,Ben_combination),3)

conf_mat_bis= confusion_matrix(Ben_combination,predicted_combination, [1, 2, 3, 4, 5, 6, 7, 8, 9] )

classes = [1, 2, 3, 4, 5, 6, 7, 8, 9] 

## Assess the bitrate

In [ ]:
from bitrate import bitrate
bitrate_minute = bitrate(acc,9)